In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 2: Install yt-dlp and ffmpeg
!pip install yt-dlp
!apt-get install -y ffmpeg

In [ ]:
# Step 3: Ask for YouTube URL
video_url = input("Enter YouTube URL: ")

In [ ]:
# Step 4: Get all available formats
import yt_dlp

In [ ]:
ydl_opts = {'quiet': True, 'skip_download': True}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(video_url, download=False)
    formats = info_dict['formats']

In [ ]:
# Step 5: Build a friendly resolution list
video_formats = []
for f in formats:
    if f.get('vcodec') != 'none':
        res = f.get('resolution') or f.get('height')
        ext = f.get('ext')
        format_id = f.get('format_id')

        # Parse resolution if it's in "WIDTHxHEIGHT" format
        if isinstance(res, str) and 'x' in res:
            try:
                res = int(res.split('x')[1])
            except:
                res = 0
        elif res is None:
            res = 0

        video_formats.append((format_id, res, ext))

In [ ]:
# Remove duplicates by resolution and sort
seen = set()
friendly_formats = []
for f in sorted(video_formats, key=lambda x: x[1]):
    if f[1] not in seen:
        friendly_formats.append(f)
        seen.add(f[1])

In [ ]:
# Step 6: Show available resolutions
print("\nAvailable video resolutions:")
for idx, (fid, res, ext) in enumerate(friendly_formats):
    print(f"{idx + 1}: {res}p ({ext}) - format code {fid}")

In [ ]:
choice = int(input("\nEnter the number for desired resolution: ")) - 1
chosen_format_id = friendly_formats[choice][0]

In [ ]:
# Step 7: Choose best audio format automatically
audio_formats = [f for f in formats if f.get('acodec') != 'none']
best_audio = max(audio_formats, key=lambda x: x.get('abr') or 0)
audio_format_id = best_audio['format_id']

In [ ]:
print(f"Selected audio format: {audio_format_id} ({best_audio.get('ext')}, {best_audio.get('abr')}kbps)")

In [ ]:
# Step 8: Set output path
output_path = "/content/drive/MyDrive/%(title)s.%(ext)s"

In [ ]:
!yt-dlp -f {chosen_format_id}+{audio_format_id} --merge-output-format mp4 -o "{output_path}" "{video_url}"

print("\nDownload completed! Check your Google Drive. Video is playable with audio.")